In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/


In [1]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
openai_api_key = os.getenv("OPENAI_PROXY_API_KEY")


In [2]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [3]:
len(text_lines)

72

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [5]:
from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key=openai_api_key, # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

/root/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02814663  0.00428726 -0.01852599  0.08190062 -0.03156214 -0.05275258
 -0.04885425  0.12481797 -0.0208328   0.03966279]


In [7]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.00578664  0.02242682 -0.01892621  0.12811586 -0.01249751 -0.07321841
 -0.00281971  0.08617394 -0.04377401  0.03073668]


In [8]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

In [9]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)


In [10]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [11]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 279620.27it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

In [12]:
question = "How is data stored in milvus?"

In [13]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [14]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.7751178741455078
    ],
    [
        "How does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float

In [15]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [16]:
context

" Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float32, Float16, and BFloat16 vector types.\n\n- Binary vectors: Store

In [17]:
question

'How is data stored in milvus?'

In [22]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [23]:
USER_PROMPT


"\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data typ

In [24]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Milvus stores data in two main categories: inserted data and metadata.

1. **Inserted Data** (including vector data, scalar data, and collection schemas) are stored in persistent storage as incremental logs. Supported storage backends include:
   - MinIO
   - AWS S3
   - Google Cloud Storage (GCS)
   - Azure Blob Storage
   - Alibaba Cloud OSS
   - Tencent Cloud Object Storage (COS)

2. **Metadata** (system-generated operational data) is stored in etcd, with each Milvus module maintaining its own metadata.

For vector data specifically:
- Binary vectors store 0/1 sequences
- Float32 is default (7-digit precision)
- Float16/BFloat16 offer reduced precision for efficiency

Data flushing occurs asynchronously - data is first queued, then written to disk as logs. The `flush()` command forces immediate disk writing.

<translated>
Milvus以两种主要类型存储数据：插入数据和元数据。

1. **插入数据**（包括向量数据、标量数据和集合模式）以增量日志形式存储在持久化存储中，支持的存储后端包括：
   - MinIO
   - AWS S3
   - 谷歌云存储(GCS)
   - Azure Blob存储
   - 阿里云OSS
   - 腾讯云

In [18]:
question = "how can i connect to AWS S3?"

In [19]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=5,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [20]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "Runtime issues\n\n  Errors that occur during runtime may cause service breakdown. To troubleshoot this issue, check compatibility between the server and your client before moving forward.\n\n\n  #",
        0.286552369594574
    ],
    [
        "What if I failed to pull the Milvus Docker image from Docker Hub?\n\nIf you failed to pull the Milvus Docker image from Docker Hub, try adding other registry mirrors. \n\nUsers from Mainland China can add the URL \"https://registry.docker-cn.com\" to the registry-mirrors array in **/etc.docker/daemon.json**.\n\n```\n{\n  \"registry-mirrors\": [\"https://registry.docker-cn.com\"]\n}\n```\n\n###",
        0.2802795171737671
    ],
    [
        "Operational FAQ\n\n<!-- TOC -->\n\n\n<!-- /TOC -->\n\n###",
        0.25892001390457153
    ],
    [
        "Can I share a MinIO instance among multiple Milvus instances?\n\nYes, you can share a MinIO instance among multiple Milvus instances. To do so, you need to change `minio.rootPath

In [21]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [22]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [23]:
SYSTEM_PROMPT

'\nHuman: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。\n'

In [24]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\nRuntime issues\n\n  Errors that occur during runtime may cause service breakdown. To troubleshoot this issue, check compatibility between the server and your client before moving forward.\n\n\n  #\nWhat if I failed to pull the Milvus Docker image from Docker Hub?\n\nIf you failed to pull the Milvus Docker image from Docker Hub, try adding other registry mirrors. \n\nUsers from Mainland China can add the URL "https://registry.docker-cn.com" to the registry-mirrors array in **/etc.docker/daemon.json**.\n\n```\n{\n  "registry-mirrors": ["https://registry.docker-cn.com"]\n}\n```\n\n###\nOperational FAQ\n\n<!-- TOC -->\n\n\n<!-- /TOC -->\n\n###\nCan I share a MinIO instance among multiple Milvus instances?\n\nYes, you can share a MinIO instance among multiple Milvus instances. To do so, you need to change `minio.rootPath` to a unique value for each Milvus instance in the conf

In [25]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文信息，其中没有包含关于如何连接到 AWS S3 的具体指导或步骤。上下文主要讨论了运行时问题、Docker 镜像拉取、MinIO 实例共享以及寻求帮助的渠道，但没有涉及 AWS S3 的连接配置。

<translated>
根据提供的上下文信息，其中没有包含关于如何连接到 AWS S3 的具体指导或步骤。上下文主要讨论了运行时问题、Docker 镜像拉取、MinIO 实例共享以及寻求帮助的渠道，但没有涉及 AWS S3 的连接配置。
</translated>
